# Import

In [ ]:
%load_ext autoreload
%autoreload 2

import importlib
import numpy as np
import math
import itertools 
import pandas as pd
import numpy as np
import io
import msoffcrypto
from sklearn.metrics import r2_score
import openpyxl
import plotly.express as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import streamlit as st
import json
from math import factorial as fact
from IPython.display import Markdown, display
import networkx as nx
import requests
import toml
import sys, os
import collections
from types import SimpleNamespace
import copy

%matplotlib inline 
%config Completer.use_jedi = False

from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

from utils import *

# eval indiv

In [ ]:
file = {'SheetMapName' : 'map', 'uploaded_file' : None}
algo = load_data_brut(file)
algo.Group = [0,2]

In [ ]:
algo.dfslot

In [ ]:
keydrop= ["confs", "dfslot","dfline","indivs","df",'A0','DataCategorie']
d = {k : v for k,v in vars(algo).items() if k not in keydrop}
s = pd.Series(d).rename('Val').astype(str)
s.index = s.index.astype(str)
s

In [ ]:
Name = 'E1-C0,E1-C1,E1-C2,E1-C3,P1-E1'
# Name  = 'E0-C1,E1-C0,E1-C2,E1-C3,P1-E0,P1-E1'
indiv = Indiv_reverse(Name,algo) 
indiv

In [ ]:
df = pd.read_excel('test.xlsx', sheet_name= 'Sheet{}'.format(5), header=0).set_index('case').T
df

# Split save

## Combinatoire

In [ ]:
SplitDict = {}
for Size in range(2,5):
    List = list(range(Size))
    a = np.array(list(itertools.product(List, repeat=Size)))
    
    mask = (a[:,0] == 0)
    for i in range(1,Size) :
        mask = mask & (a[:,i] <= a[:,:i].max(1)+1)
    a = a[mask]   
    
    for i in range(len(a)):
        s = a[i]
        u, idx, counts = np.unique(s, return_index=True, return_counts  = True, axis = 0)
        val = u[counts == 1]
        s[np.isin(s,val)] = -1 
        a[i] = s
    SplitDict[Size] = a

In [ ]:
SplitDict

In [ ]:
Group  = [0,1,2,3]
Egroup = indiv['Econnect'][1]
Egroup = [e for e in Egroup if e in Group ]
Size = len(Egroup)
Size

In [ ]:
ListIndivs = []
for s in SplitDict[Size]: 
    d = collections.defaultdict(list)
    for i in range(Size): 
        d[s[i]].append(Egroup[i])
    d = dict(sorted(d.items())) 
    NewIndiv = copy.deepcopy(indiv)
    NewIndiv['Egroup'] = Egroup
    NewIndiv['Split'] = d
    ListIndivs.append(NewIndiv)

In [ ]:
ColSysteme = ['Clist','Name','dist_Connect','List_EtoC','List_PtoE']
ColAlgo = ['CtoE','EtoP','Econnect','Elist','Ecount','Pconnect','Plist','Pcount']
Col_drop = ColSysteme +ColAlgo
pd.DataFrame(ListIndivs).drop(columns= Col_drop)

## Calcul

### Func

In [ ]:
def debit_2(algo, d_EtoC_list,d_PtoE,EClist, group = True, split = True):
    # d_EtoC = Dict_dist['EtoC']
    # d_PtoE = Dict_dist['PtoE']
    p = [-5.16e-04, -1.54e-02, 4.87]
    p = algo.Pvals
    # p = [-6.61e-4,-0.0286,12.1]
    E0 = 7.64e-04
    coef_E  = np.ones(len(EClist)) * E0
    # coef_E  = np.array([E0 if c in [0,3] else 0 for c in Clist])
    # print(Clist , coef_E)
    coef_C  = 0.036
    coef_C  = [algo.Nvals[i] for i in EClist]
    coef_C  = np.array(coef_C)
    coef_d  = 2.35e-04    
    
    A = coef_E + d_EtoC_list * coef_d + coef_C 
    Z = ( A**-0.5).sum() if group else A**-0.5
    As , Bs, Cs = p[0] - (coef_d * d_PtoE) - 1/(Z**2), p[1] , p[2]
    delta = (Bs**2) - (4 * As * Cs)
    Qt  = np.array((- Bs - delta**0.5)/(2*As))
    Pt = np.array(Qt**2 / Z**2)
    a0 = p[0] * (Qt**2) + p[1] * Qt + p[2] - Pt
    Qi = (Pt / A)**0.5
    Pi = coef_C * (Qi**2)
    key = ['Qt','Pt','Qi','Pi']
    val = [Qt, Pt, Qi, Pi]
    val = [v.round(2) for v in val]
    return dict(zip(key,val))

In [ ]:
def Calcul_Debit_2(algo ,indiv, group, split = None):
    D = algo.Comb  
    Group = algo.Group  
    Clist = D['C']
    Econnect = indiv['Econnect']
    Pconnect = indiv['Pconnect']
    EtoP = indiv['EtoP']
    Pression = []
    Debit = []

    Cpression = {}
    Cdebit = {}
    grouped = False
    split =  {0: [0], 1: [1, 2, 3]}
    for i, (e,EClist) in enumerate(Econnect.items()):
        p = EtoP[i]
        name = 'P{}-E{}'.format(p,e)
        VerifGroup = np.isin(Group,  EClist)
        # EClistTotal = [EClist]
        # if VerifGroup.all() & (len(Group) > 0):
        EClistTotal = [[i for i in EClist if i in Group], [i for i in EClist if i not in Group]]          
        grouped = True
        for j,  EClist in enumerate(EClistTotal):
            if j > 0 : grouped = False # bascule a No group apres le passage group 
            if len(EClist)>0: # bug avec calcul array
                d_EtoC_list = np.array([algo.dist['E{}-C{}'.format(e,c)] for c in EClist])
                d_PtoE = algo.dist['P{}-E{}'.format(p,e)]
                res = debit_2(algo, d_EtoC_list,d_PtoE, EClist, grouped)

                Debit = Debit + list(res['Qi'])
                Pi = list(res['Pi'])
                PressionConnect = dict(zip(EClist, Pi))
                Cpression.update(PressionConnect)                
                Qi = list(res['Qi'])
                Cdebit.update(dict(zip(EClist, Qi)))

                keys = ['i', 'j' ,'Clist','d_EtoC_list', 'd_PtoE', 'EClist', 'Group','grouped', 'EClistTotal', 'PressionConnect']
                vals = [i, j ,Clist , d_EtoC_list, d_PtoE, EClist, Group,grouped, EClistTotal, PressionConnect]
                print(str(dict(zip(keys,vals))))
    PressionList = [Cpression[i] for i in D['C']]
    DebitList    = [Cdebit[i] for i in D['C']]
    # print(Cpression)
    SumDebit = round(sum(Debit),1)   
    keys = ['PressionList','DebitList','Debit']
    vals = [PressionList, DebitList, SumDebit] 
    return dict(zip(keys,vals))

In [ ]:
split = '0, (1,2) , (3, 4)'

def Uncode_split(split):
    n = 0
    tosplit = False 
    d = collections.defaultdict(list)
    for i in split: 
        if i == '(':
            tosplit =  True
            n+= 1
        if i == ')': 
            tosplit =  False
        if i.isdigit(): 
            i = int(i)
            if tosplit: d[n].append(i)
            else : d[0].append(i)
    return d

split = '0, (1,2, 3)'
split = Uncode_split(split)
split =  {0: [0], 1: [1, 2, 3]}

### Run

In [ ]:
split = '0, (1,2, 3)'
Uncode_split(split)

In [ ]:
Nozzles = ['ramp', 'fixed', 'fixed', 'fixed']
algo.Nozzles = Nozzles
Nvals   = [algo.DataCategorie['Nozzle']['Values'][n]['a'] for n in Nozzles]
algo.Nvals = dict(zip(Clist, Nvals))
algo.Group  = [1,2]
Calcul_Debit_2(algo ,indiv, group)

In [ ]:
indiv

In [ ]:
[2.2, 4.4, 4.5, 2.0]
[36.3, 11.1, 11.1, 34.5]

In [ ]:
dc = {'i': 0, 'j': 0, 'Clist': [0, 1, 2, 3], 'd_EtoC_list': np.array([0.8 , 1.44, 0.48, 2.4 ]),
      'd_PtoE': 0.48, 'EClist': [0, 1, 2, 3], 'Group': [0, 1, 2, 3], 'grouped': True,
      'EClistTotal': [[0, 1, 2, 3], []], 'PressionConnect': {0: 1.47, 1: 2.22, 2: 2.24, 3: 2.21}}

In [ ]:
dc = {'i': 0, 'j': 0, 'Clist': [0, 1, 2, 3],
      'd_EtoC_list': np.array([1.44, 0.48]), 'd_PtoE': 0.48, 'EClist': [1, 2], 'Group': [1, 2],
      'grouped': True, 'EClistTotal': [[1, 2], [0, 3]], 'PressionConnect': {1: 4.12, 2: 4.15}}

In [ ]:
# d_EtoC = Dict_dist['EtoC']
# d_PtoE = Dict_dist['PtoE']

d_EtoC_list = dc['d_EtoC_list']
d_PtoE = dc['d_PtoE']
EClist = dc['EClist']
group = True

p = [-5.16e-04, -1.54e-02, 4.87]
p = algo.Pvals
# p = [-6.61e-4,-0.0286,12.1]
E0 = 7.64e-04
coef_E  = np.ones(len(EClist)) * E0
coef_E  = np.array([E0 if c in [0,3] else 0 for c in EClist])
coef_E = 0
# print(Clist , coef_E)
coef_C  = 0.036
coef_C  = [algo.Nvals[i] for i in EClist]
coef_C  = np.array(coef_C)
coef_d  = 2.35e-04    
coef_E , coef_C , coef_d

A = coef_E + d_EtoC_list * coef_d + coef_C 
Z = ( A**-0.5).sum() if group else A**-0.5
# Z = A**-0.5

As = p[0] - (coef_d * d_PtoE) - 1/(Z**2) - E0
Bs = p[1]
Cs = p[2]

delta = (Bs**2) - (4 * As * Cs)
Qt  = np.array((- Bs - delta**0.5)/(2*As))
Pt = np.array(Qt**2 / Z**2)
a0 = p[0] * (Qt**2) + p[1] * Qt + p[2] - Pt
Qi = (Pt / A)**0.5
Pi = coef_C * (Qi**2)
key = ['Qt','Pt','Qi','Pi']
val = [Qt, Pt, Qi, Pi]
val = [v.round(2) for v in val]
dict(zip(key,val))

## Test si multiple E slot 

In [ ]:
D = algo.Comb  
Group = algo.Group  
Clist = D['C']
Econnect = indiv['Econnect']

Pconnect = indiv['Pconnect']
EtoP = indiv['EtoP']
Pression = []
Debit = []
# Data = {}
# Pression_C = []
# on loop sur chaque EV pour connect to C et faire calcul Pt Qt coté pompe et Pi Qi coté Capteur
Cpression = {}
Cdebit = {}
grouped = False

print(Group, Econnect)
for i, (e,EClist) in enumerate(Econnect.items()):
    p = EtoP[i]
    name = 'P{}-E{}'.format(p,e)
    VerifGroup = np.isin(Group,  EClist)
    # EClistTotal = [EClist]
    # if VerifGroup.all() & (len(Group) > 0):
    EClistTotal = [[i for i in EClist if i in Group], [i for i in EClist if i not in Group]]          
    grouped = True
    for j,  EClist in enumerate(EClistTotal):
        if j >0 : grouped = False # bascule a No group apres le passage group 
        if len(EClist)>0: # bug avec calcul array
            print(i,j,e,EClist , grouped, EClist)
            if grouped : Egroup = EClist

In [ ]:
Esplit = collections.defaultdict(list)

In [ ]:
Size = len(Egroup)
Size
s = SplitDict[Size][0]
s

d = collections.defaultdict(list)
for i in range(Size): 
    d[s[i]].append(Egroup[i])
d.keys()
d

# PF ANCIEN

## func

In [ ]:
def Path1(A,start): 
    N = len(A)
    v0 = np.array([-1,1,-N,N])
    #     v0 = np.array([-1,1,-N,N, -N-1, -N+1,N+1,N-1])
    Dim = len(v0)
    e = 2
    A[start] = e
    a = A.reshape(-1)
    v = np.where(a == e)  

    while len(v) > 0 :
        v = np.tile(v, (Dim, 1)).T + v0
        v = v[np.where(a[v]==0)]
        v = np.unique(v)        
        e+=1
        a[v]=e
    return a.reshape((N,N))

def Path2(A,start,goal):
    N = len(A)
    v0 = np.array([-1,1,-N,N])
    #     v0 = np.array([-1,1,-N,N, -N-1, -N+1,N+1,N-1])
    Dim = len(v0)
    e1,e2  = A[start] , A[goal]
    a = A.reshape(-1)
    v = goal[1] + goal[0]*N
    L  = [goal]
    while e2 > 2:
        v = v + v0
        v[v > len(a)] = len(a)-1
        v = v[np.where((a[v] < e2) & (a[v] >= 2))]
        idx = a[v].argmin()
        v = v[idx]
        e2 = a[v]
        pos = (int(np.ceil(v/N)-1),  v%N)
        L.insert(0,pos)
    return L

## Run

In [ ]:
file = 'VALEO_full.tmj'
algo = load_data_brut(file)
with open(file, 'r') as f:
      data = json.load(f)
        


In [ ]:
dfline  = pd.DataFrame(data['layers'][2]['objects']).drop(columns=['rotation','width','name','height','visible','id']).rename(columns = {'class' : 'Class'})        
dfline

In [ ]:
dfslot = algo.dfslot
dfslot

In [ ]:
DictPos = {}
for idx , row in dfslot.iterrows():
    x = row.x
    y = row.y 
    ID = row.ID
    Class = row.Class
    DictPos[ID] = (y,x)
DictPos      

In [ ]:
A0 = algo.A0.copy()+1
A0[A0 == 2] = 0
for idx, row in algo.dfslot.iterrows():   
    A0[row.y, row.x] = row.Color+5
Ax = np.ones((10,10))
Ax[:A0.shape[0],:A0.shape[1]] = A0

In [ ]:
plt.imshow(Ax)

In [ ]:
Comb = dfslot.groupby('Class').Name.unique().apply(list).apply(sorted).to_dict()
Comb['C'] + Comb['P']

In [ ]:
%%time
e = 'E0'
Path = []
# fig, ax = plt.subplots(figsize = (4,4))
# f = ax.imshow(Ax)
d = {} 
for begin in ['E0','E1','E2']:
    start = DictPos[begin]
    A = Ax.copy()
    A1 = Path1(A,start)
    for end in ['C0','C1','C2','C3','P0','P1']: 

        goal = DictPos[end]
         
        
        L = Path2(A1.copy() ,start,  goal)
        L = np.array(L)
        Path.append(L)

        dist = np.abs(np.diff(L.T)).sum()
        
        if end[0] == 'C' :            ID = begin + '-' + end
        else : ID = end + '-' + begin
#         print(begin,start, end , goal)
        d[ID] = dist


In [ ]:
len(Path)
fig, ax = plt.subplots(figsize = (8,8))
masked = np.ma.masked_where(A0 <= 1, A0)
# f = ax.imshow(A0)
offset = np.linspace(-0.3,0.3,18)
for i, p in enumerate(Path):
        n = offset[i]
#         n = 0
        
        f = ax.plot(p[:,1]+n,p[:,0]+n,'w', linewidth=1)
f = ax.imshow(masked,alpha=1,cmap = 'Reds')
f = ax.imshow(A0)

In [ ]:
d

In [ ]:
Save = [DictPos, d]

In [ ]:
dfline = algo.dfline.copy()
dfline['distPF'] = dfline.ID.map(d)-0.5
dfline['distRatio'] = (dfline.distPF / dfline.dist).round(1)
dfline[['dist','distPF']].corr()
dfline

# PF avec load excel map 

In [ ]:
import matplotlib.patches as mpatch

In [ ]:
dfmap = pd.read_excel('input.xlsx', sheet_name= 'map', index_col=0)
dfmap

In [ ]:
SlotColor = {'C' : 10, 'E': 20, 'P' : 30}
A0 = dfmap.values
Size = max(A0.shape)
Comb = collections.defaultdict(list)
DictPos = {}
slots = ['C','P','E']
slotsN = dict(zip(slots,[0,0,0]))
ListBegin = []
ListEnd = []
ListWall = (np.argwhere(A0[1:-1,1:-1] == 1)+1).tolist()

for iy, ix in np.ndindex(A0.shape):
    v = A0[iy, ix]
    if type(v) == str: 
        slot = v[0]
        n = slotsN[slot]
        slotsN[slot] = n+1
        v = slot + str(n)
        A0[iy,ix] = SlotColor[slot]*20
        #Comb[v[0]].append(int(v[1:]))
        Comb[v[0]].append(int(n))
        DictPos[v] = (iy,ix)
        
        if slot == "E" : ListBegin.append(v)
        else : ListEnd.append(v)  
            
A0 = A0.astype(float)      
Comb
Ax = np.ones((Size,Size))
Ax[:A0.shape[0],:A0.shape[1]] = A0
DictPos
ListBegin, ListEnd = sorted(ListBegin), sorted(ListEnd)
ListBegin, ListEnd 


In [ ]:
ListEnd = ['E1', 'E2','E0', 'E2','E0', 'E1']
ListBegin, ListEnd 

In [ ]:
fig, ax = plt.subplots()
# im = ax.imshow(Aplot, cmap=plt.get_cmap('Accent'))
im = ax.imshow(Ax)
fig.colorbar(im)

In [ ]:
%%time
# e = 'E0'
Path = {}
d = {} 
DictLine = {}

for begin in ListBegin:
    start = DictPos[begin]
    A = Ax.copy()
    A1 = Path1(A,start)
    for end in ListEnd: 
        goal = DictPos[end]        
        path = Path2(A1.copy() ,start,  goal)
        path = np.array(path)       
        dist = np.abs(np.diff(path.T)).sum()
        
        if end[0] == 'C' : ID = begin + '-' + end
        else : ID = end + '-' + begin
        #print(begin,start, end , goal)
#         Path.append(L)
        DictLine[ID] = {'path' : path, 'dist' : dist}
#         d[ID] = dist

In [ ]:
DictLine

In [ ]:
DictDistEtoE = pd.DataFrame(DictLine).T.drop(columns = 'path').to_dict()['dist']

In [ ]:
LenPath = len(DictLine)
Ymax , Xmax = A0.shape
PlotColor = {'C' : "#93c9ee", 'E': '#a2ee93', 'P' : "#c593ee"}
fig, ax = plt.subplots(figsize = (8,8))

f = ax.add_patch(mpatch.Rectangle((0,0), Xmax-1, Ymax-1, color='#d8d8d8'))
# masked = np.ma.masked_where(A0 <= 1, A0)
offset = np.linspace(-0.35,0.35,LenPath)
for i, (slot,data) in enumerate(DictLine.items()):
    n = offset[i]  
    p = data['path']
    if slot[0] == 'E' : 
        f = ax.plot(p[:,1]+n,p[:,0]+n,"#32cdff", linewidth=1, zorder=1, linestyle ='-')
    else : 
        f = ax.plot(p[:,1]+n,p[:,0]+n,"#3286ff", linewidth=2, zorder=1, linestyle ='-')

style = dict(size=15, color='black')
for slot, pos in DictPos.items(): 
    x , y = pos
    Type = slot[0]
    color = PlotColor[Type]
    f = ax.add_patch(mpatch.Rectangle((y-0.4,x-0.4), 0.8, 0.8, color=color))
    f = ax.add_patch(mpatch.Rectangle((y-0.4,x-0.4), 0.8, 0.8, color='black', fill = None))
    f = ax.text(y, x+0.1,slot , **style,  ha='center', weight='bold') 
#     f = ax.add_patch(mpatch.Rectangle((y-0.5,x-0.5), 1, 1, color='green'))

for x,y in ListWall: 
    f = ax.add_patch(mpatch.Rectangle((y-0.5,x-0.5), 1, 1, color='black'))
f = ax.imshow(np.zeros(A0.shape), cmap='gray',vmin=0,vmax=1)


In [ ]:
d

In [ ]:
d

In [ ]:
len(Path)
fig, ax = plt.subplots(figsize = (8,8))
masked = np.ma.masked_where(A0 <= 1, A0)
# f = ax.imshow(A0)
offset = np.linspace(-0.3,0.3,18)
for i, p in enumerate(Path):
        n = offset[i]
#         n = 0
        
        f = ax.plot(p[:,1]+n,p[:,0]+n,'w', linewidth=1)
# f = ax.imshow(masked,alpha=1,cmap = 'Reds')
f = ax.imshow(A0)

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.patches as mpatch

overlap = {name for name in mcolors.CSS4_COLORS
           if f'xkcd:{name}' in mcolors.XKCD_COLORS}

In [ ]:
overlap

# Run algo

In [ ]:
file = {'SheetMapName' : 'map', 'uploaded_file' : "input.xlsx"}

algo = load_data_brut(file)
algo.df = indiv_init(algo, 10)

In [ ]:
algo.dfline

In [ ]:
algo.GroupDict = {0: 1,
                  1: 1, 
                  2: 0,
                  3: 2}

In [ ]:
algo.DataCategorie

In [ ]:
algo.PompesSelect = ['Pa','Pa']

In [ ]:
Name =  'E0-C1,E1-C2,E2-C0,E2-C3,P0-E1,P1-E0,P1-E2'
Name = "E1-C0,E1-C1,E1-C2,E1-C3,P1-E1"
Name = "E0-C0,E1-C2,E2-C1,E2-C3,P0-E1,P0-E2,P0-E0"
# Name = "E0-C0,E0-C2,E1-C1,E1-C3,P0-E1,P0-E0"
indiv = Indiv_reverse(Name,algo) 
indiv

In [ ]:
ElemsList = ['Clist','Elist','Plist']
Elems = ['C','E','P']
SelectSlot = []
List_EtoC = indiv['List_EtoC']
List_PtoE = indiv['List_PtoE']
for n in range(3):
    SelectSlot+= ['{}{}'.format(Elems[n],i) for i in indiv[ElemsList[n]]]
SelectLine = indiv['Name']    
fig = new_plot(algo, SelectLine, SelectSlot)

In [ ]:
distPerLine = np.array([algo.dist[line] for line in indiv['List_PtoE'][0]])
distPerLine

In [ ]:
DictDistEtoE
np.array([algo.dist[line] for line in indiv['List_PtoE'][0]])

In [ ]:
SelectLine = algo.DictLine.keys()
SelectSlot = algo.DictPos.keys()

In [ ]:
SelectLine = algo.DictLine.keys()
SelectSlot = algo.DictPos.keys()
fig = new_plot(algo, SelectLine, SelectSlot)

# P2

In [ ]:
algo.GroupDict = {0: 1, 1: 1, 2: 0, 3: 2}

In [ ]:
Econnect = indiv['Econnect']
Econnect


# Group

In [ ]:
algo.GroupDict = {0: 1, 1: 1, 2: 0, 3: 2}

In [ ]:
D = algo.Comb  
gr = algo.GroupDict 
Clist = D['C']
Econnect = indiv['Econnect']
Pconnect = indiv['Pconnect']
EtoP = indiv['EtoP']
Ptype = indiv['Ptype']
Pression = []
Debit = []
Cpression = {}
Cdebit = {}
grouped = False
gr

In [ ]:
for i, (e,EClist) in enumerate(Econnect.items()):
    p = EtoP[i]
    pt = Ptype[i]
    name = 'P{}-E{}'.format(p,e)
    VerifGroup = np.isin(Group,  EClist)
#     print(e,EClist, p,pt, Group)
    d = collections.defaultdict(list)
    for j, c in enumerate(EClist): 
        d[gr[c]].append(c)
    for g,Clist in d.items():
        if g == 0 : grouped = False
        else :grouped = True
#         print(g,Clist)
        d_EtoC_list = np.array([algo.dist['E{}-C{}'.format(e,c)] for c in Clist])
        d_PtoE = algo.dist['P{}-E{}'.format(p,e)]
        res = debit(algo, d_EtoC_list,d_PtoE, Clist,pt, grouped, split = False)

        Debit = Debit + list(res['Qi'])
        Pi = list(res['Pi'])
        PressionConnect = dict(zip(Clist, Pi))
        Cpression.update(PressionConnect)

        Qi = list(res['Qi'])
        Cdebit.update(dict(zip(Clist, Qi)))

        # Data[name] = res        
        # Pression = Pression + list(res['Pi'])          
        # print(dc,dp,Clist,list(res['Pi']))
        # Pression_C = Pression_C + [PressionConnect]
        print(i, j , g,Clist, grouped ,EClist,Clist, PressionConnect)
    

# MAsse cout 

In [ ]:
indiv['Name']

In [ ]:
dmasse = {}
dcout = {}
# confs = algo.confs

for Categorie in ['Pompe', 'Tuyau','EV']:
    v = algo.DataCategorie[Categorie]['Values']
    if Categorie == 'Pompe' : 
        Ptype = indiv['Ptype']
        dmasse[Categorie] = int(sum([algo.DataCategorie[Categorie]['Values'][pt]['Masse'] for pt in Ptype]))
        dcout[Categorie]  = int(sum([algo.DataCategorie[Categorie]['Values'][pt]['Cout']  for pt in Ptype]))
    if Categorie == 'Tuyau' :
#         Factor = indiv['dist']
#         Name = algo.Tuyau
        distPerLine = np.array([algo.dist[line] for line in indiv['Name']])
        MassePerLine = np.array([v[algo.duriteType[line]]['Masse'] for line in indiv['Name']])
        CoutPerLine  = np.array([v[algo.duriteType[line]]['Cout'] for line in indiv['Name']])
        dmasse[Categorie] = (distPerLine * MassePerLine).sum()
        dcout[Categorie] = (distPerLine * CoutPerLine).sum()
    if Categorie == 'EV' :
        Ccount = len(algo.Comb['C'])
        Factor = Ccount
        Name = algo.EV  
        dmasse[Categorie] = int(sum([Factor * v[n]['Masse'] for n in Name]))
        dcout[Categorie]  = int(sum([Factor * v[n]['Cout']  for n in Name]))
        
dmasse['Reservoir'] = 600
dcout['Reservoir']  = 30  
info = [dmasse, dcout]
Masse = round(sum(dmasse.values()),2)
Cout = round(sum(dcout.values()),2)
info, { 'Masse' : Masse, 'Cout' : Cout}

# Pompes

In [ ]:
file = {'SheetMapName' : 'map', 'uploaded_file' : None}

algo = load_data_brut(file)
algo.df = indiv_init(algo, 10)

In [ ]:
ListPompes = algo.DataCategorie['Pompe']['Unique']
dict(zip(ListPompes, [2]*len(ListPompes)))

In [ ]:
algo.DataCategorie

In [ ]:
Name =  'E0-C1,E1-C2,E2-C0,E2-C3,P0-E1,P1-E0,P1-E2'
indiv = Indiv_reverse(Name,algo) 
indiv

In [ ]:
Plist = indiv['Plist']

In [ ]:
EcMax = 2
NewCtoE = np.array([0,1,2,0])
NewCtoE
Elist = np.unique(NewCtoE)
Elist
Ecount = len(Elist)
Ecount > EcMax
n = Ecount -  EcMax 
Edrop = np.random.choice(Elist,n, replace=False)
Edrop
Edispo = Elist[~np.isin(Elist, Edrop)]
Edispo
mask = np.isin(NewCtoE,Edrop)
mask
NewCtoE[mask] = np.random.choice(Edispo,mask.sum())
NewCtoE

In [ ]:
prime_numbers = [2, 3, 5, 9, 9, 11]
prime_numbers.remove(9)
prime_numbers

In [ ]:
EcMax = 2
NewCtoE = np.array([0,1,2,0])
Elist = np.unique(NewCtoE)
Ecount = len(Elist)
n = Ecount -  EcMax 
if n > 0:
    Edrop = np.random.choice(Elist,n, replace=False)
    Edispo = Elist[~np.isin(Elist, Edrop)]
    mask = np.isin(NewCtoE,Edrop)
    NewCtoE[mask] = np.random.choice(Edispo,mask.sum())
NewCtoE

In [ ]:
mask = np.isin(NewCtoE,Edrop)
mask
NewCtoE[mask] = np.random.choice(Edispo,mask.sum())
NewCtoE

In [ ]:
idx = 0
p = 

In [ ]:
arr = range(5)
arr
# numbers below will never contain repeated numbers (replace=False)
numbers = np.random.choice(arr, 3, replace=False) 
numbers

In [ ]:
Calcul_Debit(algo ,indiv, split = False)

## plot

In [ ]:
# SelectLine = algo.DictLine.keys()
# SelectSlot = algo.DictPos.keys()

row = algo.df.iloc[0]
                        
ElemsList = ['Clist','Elist','Plist']
Elems = ['C','E','P']
SelectSlot = []
List_EtoC = row.List_EtoC
List_PtoE = row.List_PtoE
for n in range(3):
    SelectSlot+= ['{}{}'.format(Elems[n],i) for i in row[ElemsList[n]]]
SelectLine = row.Name

fig = new_plot(algo, SelectLine, SelectSlot)